# OC Lab — end-to-end demo


## Run the cell below to install dependencies

In [6]:
!pip install -e ../

Obtaining file:///Users/troy/Downloads/oclab
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for oclab (pyproject.toml) ... done
  Created wheel for oclab: filename=oclab-0.1.0-0.editable-py3-none-any.whl size=1583 sha256=36f474f5ecf77e7915d3dbb7a5c73a373a114078be7c07f38a307ff95486afa0
  Stored in directory: /private/var/folders/9r/kdm1pd5j7w99kt_xjm2zm5sh0000gn/T/pip-ephem-wheel-cache-z99nrg8k/wheels/8b/c8/56/29dd95b70c65fc357b9703a36c20f2d3dd08db5e6f0a3635cd
Successfully built oclab
  Attempting uninstall: oclab
    Found existing installation: oclab 0.1.0
    Uninstalling oclab-0.1.0:
      Successfully uninstalled oclab-0.1.0


### End-to-End OC Harness

This cell runs the **entire OC pipeline**—from Fisher-geometry through Standard-Model RG evolution and the toy FRG flow

---

#### What the harness does
1. **Geometry → Couplings**  
   Monte-Carlo evaluates the Fisher-metric “stiffness” integrals \(I_a\) for U(1), SU(2), SU(3) using the centers and metric settings in the config.  
   From these and the EW observables (αₑₘ, sin²θ₍W₎) it computes the OC scale `x` and the gauge couplings at the UV anchor μ₀.

2. **RG running → MZ**  
   Evolves α₁, α₂, α₃ from μ₀ to 91.1876 GeV with the two-loop SM β-functions (Q3 thresholds).  
   The report compares the resulting α₃(MZ) to the PDG empirical value **0.1181** and flags whether it agrees within ≈ 1 %.

3. **FRG flow → k\***  
   Integrates a toy background-field flow  
   \( \dot\alpha = c\,\alpha^2 \)  
   and stops when |η(α)| = η_freeze (≈ 0.99).  
   It prints both the **numeric** freeze scale k\* and the **analytic expectation** from the ODE solution, checking their log-difference < 0.15.  
   The expected “mid-IR” range is **0.3 – 1.2 GeV**; being in this band is plausible for an onset scale.

4. **OC mass scale & ratio**  
   Computes \(m_\tau\) (internal OC normalization) and the dimensionless ratio  
   \( \mathcal{C} = k_\*/m_\tau \).  
   𝓒 currently serves as a comparative diagnostic, not an external prediction.

5. **Reporting & saving**  
   The cell prints:
   - geometry and MC statistics  
   - EW-lock couplings  
   - α₃(MZ) vs PDG (% difference + status)  
   - FRG parameters and k\* checks  
   - mτ and 𝓒  
   It then saves a timestamped JSON bundle with all numeric and analytic values for provenance.

---

#### How to interpret the report
- ✅ **α₃(MZ ≈ 0.118 ± 1 %)** → SM normalization consistent with experiment.  
- ✅ **η-based stop + small ln-error (< 0.15)** → FRG integration behaving as expected.  
- ✅ **k\* in 0.3–1.2 GeV** → plausible mid-IR freeze scale.  
- 𝓒 ≈ 0.04–0.06 → typical internal ratio for current OC normalization.

⚠️ If any status shows *CHECK*, revisit the corresponding configuration:
- α₃ mismatch → adjust `Ka_group_scale['SU3']`.
- wrong freeze reason → set `alpha_target` ≫ α_cap.
- k\* far outside mid-IR → tweak `growth_c` or η model.

This single cell gives a full empirical vs. model sanity check for a given OC configuration.


In [7]:
# --- End-to-end OC harness report ---

import yaml, math, time
import numpy as np

from oclab.config import GeometryConfig, EWAnchor, OCParams, RGRun, FRGScan
from oclab.pipeline import geometry_to_couplings, oc_gap_from_frg
from oclab.frg.flows import expected_kstar_toy
from oclab.io.datasets import bundle_results

# ----------------------- Settings -----------------------
CFG_PATH = "../configs/default.yaml"
PDG_ALPHA3_MZ = 0.1181                 # empirical target for αs(MZ)
MID_IR_BAND   = (0.3, 1.2)             # “plausible” mid-IR k* window (GeV) for toy η

# -------------------- Helper functions ------------------
def pct_diff(meas, ref):
    return 100.0 * (float(meas) - float(ref)) / float(ref)

def status(ok, ok_msg="OK", bad_msg="CHECK"):
    return ("✅ " + ok_msg) if ok else ("⚠️ " + bad_msg)

def fmt(v, prec=6):
    return f"{float(v):.{prec}g}"

# ---------------------- Load config ---------------------
cfg = yaml.safe_load(open(CFG_PATH))
g   = GeometryConfig(**cfg["geometry"])
ew  = EWAnchor(**cfg["ew_anchor"])
oc  = OCParams(**cfg["oc_params"])
rg  = RGRun(**cfg["rg"])
frg = FRGScan(**cfg["frg_scan"])

# ----------------- Geometry → Couplings -----------------
t0 = time.time()
calib, hist = geometry_to_couplings(g, ew, oc, rg)
t1 = time.time()

# RG landing at MZ (final entry):
alpha3_MZ = float(hist.alpha3[-1])
alpha3_mu0 = float(calib.alphas_mu0["SU3"])
delta_pct_alpha3 = pct_diff(alpha3_MZ, PDG_ALPHA3_MZ)
ok_alpha3 = abs(delta_pct_alpha3) <= 1.0  # within ~1% of PDG

# ------------------------ FRG ---------------------------
frg_res, m_tau, C = oc_gap_from_frg(frg, alpha3_mu0, ew.mu0, oc, calib.x)
k_star_num = frg_res.k_star
k_star_exp = expected_kstar_toy(alpha3_mu0, frg.growth_c, frg.kmax, frg.eta_freeze, model=frg.model)
ln_err = abs(math.log(k_star_exp / k_star_num)) if (k_star_num and k_star_exp) else float("inf")

ok_eta_reason = (getattr(frg_res, "reason", None) == "eta_freeze")
ok_ln_match   = (ln_err < 0.15)  # toy ODE vs numeric should agree within ~15% in log-space
ok_k_band     = (MID_IR_BAND[0] <= k_star_num <= MID_IR_BAND[1])

t2 = time.time()

# ----------------------- Report -------------------------
print("OC Harness Report")
print("──────────────────")
print(f"Config     : {CFG_PATH}")
print(f"Timestamp  : {time.strftime('%Y-%m-%d %H:%M:%S')}")
print()

print("Inputs / Anchors")
print("  • μ0 (UV anchor)           :", fmt(ew.mu0), "GeV   (scheme:", ew.scheme, ")")
print("  • α_em(μ0), sin²θW(μ0)     :", fmt(ew.alpha_em_mu0), ",", fmt(ew.sin2_thetaW_mu0))
print()

print("Geometry (Fisher MC) → stiffness I_a and scale x")
print("  • MC samples, seed         :", g.n_samples, ",", g.rng_seed)
print("  • metric model, jitter     :", g.metric_model, ",", g.jitter)
print("  • I_U1, I_SU2, I_SU3       :", fmt(calib.Ia['U1']), ",", fmt(calib.Ia['SU2']), ",", fmt(calib.Ia['SU3']))
print("  • I errors (SE)            :", fmt(calib.Ia_err['U1']), ",", fmt(calib.Ia_err['SU2']), ",", fmt(calib.Ia_err['SU3']))
print("  • x (OC scale)             :", fmt(calib.x))
print()

print("EW lock → couplings at μ0")
print("  • α1(μ0), α2(μ0), α3(μ0)   :",
      fmt(calib.alphas_mu0["U1"]), ",", fmt(calib.alphas_mu0["SU2"]), ",", fmt(alpha3_mu0))
print()

print("RG running → MZ")
print("  • μ_final                  :", fmt(hist.mu[-1]), "GeV (should be 91.1876 GeV)")
print("  • α3(MZ)                   :", fmt(alpha3_MZ),
      f"(target {PDG_ALPHA3_MZ} → {delta_pct_alpha3:+.2f}% )",
      status(ok_alpha3, "α3@MZ ≈ PDG", "α3@MZ off PDG"))
print()

print("FRG (background-field toy, η-based freeze)")
print("  • growth_c, η_freeze, α_cap:", fmt(frg.growth_c), ",", fmt(frg.eta_freeze), ",", fmt(frg.alpha_cap))
print("  • k* numeric (reason)      :", fmt(k_star_num), "GeV", f"({getattr(frg_res,'reason',None)})",
      status(ok_eta_reason, "η-based stop", "stopped by α-target"))
print("  • k* analytic (toy ODE)    :", fmt(k_star_exp), "GeV  (ln|k*_num/k*_exp|=", f"{ln_err:.3f}", ")",
      status(ok_ln_match, "ODE~numeric match", "mismatch"))
print("  • mid-IR plausibility      :", status(ok_k_band, f"in {MID_IR_BAND[0]}–{MID_IR_BAND[1]} GeV",
                                                f"outside {MID_IR_BAND[0]}–{MID_IR_BAND[1]} GeV"))
print()

print("OC mass scale & ratio")
print("  • m_tau                    :", fmt(m_tau), "GeV  (internal OC normalization)")
print("  • 𝓒 = k*/m_tau             :", fmt(C), " (internal diagnostic)")
print()

print("Timing")
print(f"  • geometry+RG: {(t1 - t0):.2f}s   • FRG: {(t2 - t1):.2f}s   • total: {(t2 - t0):.2f}s")
print()

# Write a results bundle with analytic k* recorded
bundle_results({
    "x": calib.x,
    "Ia": calib.Ia,
    "Ia_err": calib.Ia_err,
    "alphas_mu0": calib.alphas_mu0,
    "final_mu": float(hist.mu[-1]),
    "alpha3_final": float(hist.alpha3[-1]),
    "k_star": k_star_num,
    "m_tau": m_tau,
    "C": C,
    "frg": {
        "k_star": k_star_num,
        "reason": getattr(frg_res, "reason", None),
        "reached_alpha": getattr(frg_res, "reached_alpha", None),
        "reached_eta": getattr(frg_res, "reached_eta", None),
        "expected_k_star": k_star_exp,
        "ln_error_expected_vs_numeric": ln_err,
    },
}, prefix="notebook_run_report")


OC Harness Report
──────────────────
Config     : ../configs/default.yaml
Timestamp  : 2025-10-29 09:52:53

Inputs / Anchors
  • μ0 (UV anchor)           : 150 GeV   (scheme: Q3 )
  • α_em(μ0), sin²θW(μ0)     : 0.00782 , 0.2312

Geometry (Fisher MC) → stiffness I_a and scale x
  • MC samples, seed         : 160000 , 7777
  • metric model, jitter     : hessian_logV , 1e-09
  • I_U1, I_SU2, I_SU3       : 1 , 0.509011 , 0.997373
  • I errors (SE)            : 4.69754e-18 , 0.00307741 , 0.00117437
  • x (OC scale)             : 75.5667

EW lock → couplings at μ0
  • α1(μ0), α2(μ0), α3(μ0)   : 0.0169528 , 0.0338235 , 0.109526

RG running → MZ
  • μ_final                  : 91.1876 GeV (should be 91.1876 GeV)
  • α3(MZ)                   : 0.118098 (target 0.1181 → -0.00% ) ✅ α3@MZ ≈ PDG

FRG (background-field toy, η-based freeze)
  • growth_c, η_freeze, α_cap: 1.65 , 0.99 , 5
  • k* numeric (reason)      : 0.749823 GeV (eta_freeze) ✅ η-based stop
  • k* analytic (toy ODE)    : 0.759629 GeV 

'/Users/troy/Downloads/oclab/notebooks/results/notebook_run_report_20251029-095253.json'